### Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import json

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

%matplotlib inline
sys.path.append("..")

from utils import label_map_util
from utils import visualization_utils as vis_util
from utils import path_detection_utils as path_util
from utils import data_collection_util as data_util

### Loading in model 

In [2]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

### Object Detection on specific frame

In [3]:
path1 = 'video_frames/frame300.jpg'
path2 = 'video_frames/frame301.jpg'
path3 = 'video_frames/frame302.jpg'


image1 = Image.open(path1)
width1, height1 = image1.size
image_np1 = path_util.load_image_into_numpy_array(image1)
boxes1, scores1, classes1 = path_util.get_segmentation(path1,detection_graph, category_index, False)
human_boxes1, human_scores1 = path_util.remove_non_human(boxes1, scores1, classes1)
good_boxes1, good_scores1 = path_util.remove_low_prob(human_boxes1, human_scores1)
good_boxes1, good_scores1 = path_util.remove_poorly_sized_people(good_boxes1, good_scores1)

image2 = Image.open(path2)
width2, height2 = image2.size
image_np2 = path_util.load_image_into_numpy_array(image2)
boxes2, scores2, classes2 = path_util.get_segmentation(path2,detection_graph, category_index, False)
human_boxes2, human_scores2 = path_util.remove_non_human(boxes2, scores2, classes2)
good_boxes2, good_scores2 = path_util.remove_low_prob(human_boxes2, human_scores2)
good_boxes2, good_scores2 = path_util.remove_poorly_sized_people(good_boxes2, good_scores2)

image3 = Image.open(path3)
width3, height3 = image3.size
image_np3 = path_util.load_image_into_numpy_array(image3)
boxes3, scores3, classes3 = path_util.get_segmentation(path3,detection_graph, category_index, False)
human_boxes3, human_scores3 = path_util.remove_non_human(boxes3, scores3, classes3)
good_boxes3, good_scores3 = path_util.remove_low_prob(human_boxes3, human_scores3)
good_boxes3, good_scores3 = path_util.remove_poorly_sized_people(good_boxes3, good_scores3)

IOError: [Errno 2] No such file or directory: 'video_frames/frame300.jpg'

In [4]:
path_util.draw_box(good_boxes1[0], width1, height1, image_np1)

NameError: name 'good_boxes1' is not defined

In [3]:
#stuff = data_util.collect_data('video_frames/frame', 0, 5000,5002, detection_graph, category_index)
stuff = data_util.collect_data('video_frames/frame', 0, 9600,9600, detection_graph, category_index)

utils/data_collection_util.py:167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if flow == "nah":
utils/data_collection_util.py:191: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if flow == "nah":
utils/data_collection_util.py:222: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if vect == "nah":


In [4]:
count_diff = 0
count_total = 0
for i in range(len(stuff[0])):
    thing = stuff[0][i]
    if thing['direction'] is not thing['direction2']:
        count_diff+=1
    count_total+=1
print(count_diff)
print(count_total)    

1141
41362


In [5]:
with open('T_1_labels.json', 'w') as fp:
    json.dump(stuff[1], fp)

In [6]:
stuff2 = []
for i in stuff[0]:
    entry = i
    entry['box'] = entry['box'].tolist()
    #entry['flow'] = entry['flow'].tolist()
    try:
        entry['flow'] = [float(i) for i in entry['flow']]
        entry['flow2'] = [float(i) for i in entry['flow2']]
    except:
        entry['flow'] = [-10.0, -10.0]
        entry['flow2'] = [-10.0, -10.0]
    stuff2.append(entry)

In [7]:
with open('T_1_everything.json', 'w') as fp:
    json.dump(stuff2, fp)

In [8]:
with open('T_1_everything.json') as data_file:    
    all_data = json.load(data_file)

In [9]:
with open('T_1_labels.json') as data_file:    
    all_data_labels = json.load(data_file)

In [10]:
person_to_boxes = defaultdict(list)
person_to_flow = defaultdict(list)
person_to_flow_x = defaultdict(list)
for entry in all_data:
    if all_data_labels[str(entry['person_id'])] != "unknown":
        person_to_boxes[entry['person_id']].append(entry['box'])
        person_to_flow[entry['person_id']].append((entry['direction'], entry['direction2']))
        person_to_flow_x[entry['person_id']].append((entry['flow'][0], entry['flow2'][0]))

In [11]:
correct = 0
total = 0
incorrect1 = []
for i in person_to_boxes:
    label = all_data_labels[str(i)][0]
    box = person_to_boxes[i][0]
    center_x = (box[3]+box[1])/2.0
    if center_x < 0.5 and label == "l":
        correct+=1
    elif center_x >=0.5 and label == "r":
        correct+=1
    else:
        incorrect1.append(i)
    total+=1
print(correct/ (1.0*total))
print(total)
print(incorrect1)

0.840375586854
213
[2561, 3072, 2563, 2647, 3330, 2575, 52, 2642, 2659, 2165, 2331, 2218, 2235, 3058, 2649, 2351, 2192, 1897, 2413, 2942, 1004, 1005, 75, 3012, 2925, 2512, 2515, 2526, 3042, 489, 1448, 498, 2550, 3071]


In [12]:
correct = 0
total = 0
incorrect2 = []
for i in person_to_flow:
    label = all_data_labels[str(i)][0]
    if person_to_flow_x[i][0][0] <= 0 and label == "l":
        correct+=1
    elif person_to_flow_x[i][0][0] > 0 and label == "r":
        correct+=1
    else:
        incorrect2.append(i)
    total+=1
print(correct/ (1.0*total))
print(incorrect2)

0.460093896714
[2561, 3072, 2563, 2565, 3330, 2575, 1569, 52, 3599, 2655, 3937, 72, 3658, 1611, 596, 599, 2651, 2652, 1631, 2658, 1635, 2662, 2664, 3669, 2165, 632, 635, 2172, 2174, 642, 2184, 650, 655, 656, 1147, 2196, 2331, 2247, 177, 189, 704, 707, 708, 709, 711, 715, 716, 717, 212, 725, 215, 729, 227, 247, 3324, 3326, 258, 261, 3855, 795, 2855, 2351, 2365, 2370, 2373, 3405, 2384, 2393, 2398, 3557, 1897, 2922, 3948, 2413, 3953, 3957, 3963, 3964, 2942, 3934, 3976, 1005, 405, 3481, 2972, 2973, 418, 2979, 420, 422, 2983, 424, 2985, 426, 2987, 429, 2994, 2996, 2997, 439, 1977, 3006, 2512, 2515, 2965, 2526, 4065, 3042, 3560, 489, 495, 1448, 498, 2550, 3071]


In [13]:
person_to_flow

defaultdict(list,
            {0: [(u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left')],
             9: [(u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left')],
             21: [(u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left')],
             23: [(u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
              (u'left', u'left'),
             

In [14]:
person_to_flow_x[i]

[(2.9246461391448975, 6.078494071960449)]

In [15]:
person_to_flow[i]

[(u'right', u'right')]

In [16]:
correct = 0
total = 0
incorrect2 = []
for i in person_to_flow:
    label = stuff[1][i][0]
    if person_to_flow_x[i][0] <= 0 and label == "l":
        correct+=1
    elif person_to_flow_x[i][0] > 0 and label == "r":
        correct+=1
    else:
        incorrect2.append(i)
    total+=1
print(correct/ (1.0*total))
print(incorrect2)

0.399061032864
[0, 2561, 3072, 2563, 2565, 9, 2647, 3330, 2575, 2068, 21, 23, 2073, 1569, 3053, 2095, 2102, 57, 60, 3652, 69, 3937, 73, 3658, 1611, 81, 2642, 2659, 596, 85, 599, 89, 1631, 96, 1635, 3669, 2165, 2172, 2174, 642, 2184, 4072, 1147, 2196, 2331, 2215, 2218, 2247, 181, 2235, 704, 3058, 707, 708, 709, 2649, 711, 715, 716, 717, 1742, 725, 729, 227, 247, 258, 261, 2351, 2365, 2370, 2373, 2108, 2384, 2393, 2398, 2192, 1897, 3948, 2413, 3953, 3957, 3963, 3964, 2942, 3934, 3976, 1004, 1005, 2972, 2973, 418, 2979, 420, 422, 2983, 424, 2985, 426, 2987, 429, 1455, 2994, 2996, 2997, 439, 1977, 3006, 75, 1479, 2512, 2515, 2965, 986, 2526, 4065, 3042, 1509, 489, 1516, 1517, 2032, 1448, 498, 2037, 2550, 2041, 1450, 3071]


In [17]:
s1 = set(incorrect1)
s2 = set(incorrect2)
s1&s2

{75,
 489,
 498,
 1004,
 1005,
 1448,
 1897,
 2165,
 2192,
 2218,
 2235,
 2331,
 2351,
 2413,
 2512,
 2515,
 2526,
 2550,
 2561,
 2563,
 2575,
 2642,
 2647,
 2649,
 2659,
 2942,
 3042,
 3058,
 3071,
 3072,
 3330}